In [ ]:
import polars as pl
import pandas as pd
import preprocessing.feature_engineering as fe
import csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score
from sklearn.preprocessing import StandardScaler

In [ ]:
RSEED = 42

In [ ]:
#df_selected = pl.read_parquet("./data/2022_data_selected.parquet")

In [ ]:
"""
df_st4000 = df_selected.filter((pl.col("model") == "ST4000DM000"))
st400_dict = fe.create_faildate_dict(df_st4000)
df_st4000 = fe.create_faildate(df_st4000, st400_dict)
df_st4000 = fe.create_target_classification(df_st4000)


df_st4000.write_parquet("./data/2022_data_ST4000DM000.parquet")
"""

In [ ]:
df = pl.read_parquet("./data/2022_data_ST4000DM000.parquet")

filter out faulty data entries:

In [ ]:
faulty_data = df.filter(pl.col('Time_till_failure') < 0 )['serial_number'].unique()
faulty_data = faulty_data.to_list()

In [ ]:
df_filter = df.filter( ~ pl.col("serial_number").is_in(faulty_data))

In [ ]:
print(df.shape)
print(df_filter.shape)

In [ ]:
df_filter = df_filter.to_pandas()

In [ ]:
df_basemodel = df_filter.loc[:,["smart_5_raw","smart_187_raw","smart_188_raw","smart_197_raw","smart_198_raw","failing_in14days"]]

In [ ]:
df_basemodel = df_basemodel.dropna()

In [ ]:
df_basemodel.shape

In [ ]:
df_target = df_basemodel.pop("failing_in14days")

In [ ]:
df_target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_basemodel, df_target, train_size= 0.7, random_state=42, stratify=df_target)

In [ ]:
st_scaler = StandardScaler()

In [ ]:
X_train = st_scaler.fit_transform(X_train)

In [ ]:
X_test = st_scaler.transform(X_test)

In [ ]:
basemodel = LogisticRegression()

In [ ]:
basemodel.fit(X_train, y_train)

In [ ]:
y_pred = basemodel.predict(X_test)

In [ ]:
train_pred = basemodel.predict(X_train)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
matrix = confusion_matrix(y_test, y_pred)

In [ ]:
recall_score(y_test, y_pred)

In [ ]:
matrix

In [ ]:
train_matrix = confusion_matrix(y_train, train_pred)
train_matrix

### basemodel with if-else statements

base model (from backblaze): 
    if smart 5 > 0 OR smart 187 > 0 OR smart 188 > 0 OR smart 197 > 0 OR smart 198 >0:
        hard drive fails
    else: not failing

In [ ]:
df_S3010LL5 = df_selected.filter(pl.col("serial_number")=="S3010LL5")
df_sorted2 = df_S3010LL5.sort(by='date')



In [ ]:
df_ZJV3BYAY = df_selected.filter(pl.col("serial_number")=="ZJV3BYAY")
df_sorted1 = df_ZJV3BYAY.sort(by='date')

In [ ]:
def basemodel(df, threshold):
    df = df.to_pandas()
    fail_pred = []
    for index, row in df.iterrows():
        counter = 0
        if row["smart_5_raw"] > 0:
            counter +=1
        if row["smart_187_raw"] > 0:
            counter +=1
        if row["smart_188_raw"] > 0:
            counter +=1
        if row["smart_197_raw"] > 0:
            counter +=1           
        if row["smart_198_raw"] > 0:
            counter +=1
        if counter >= threshold:
            fail_pred.append(1)
        else:
            fail_pred.append(0)
    return fail_pred


test basemodel S3010LL5


In [ ]:
y_pred_S3010LL5_t2 = basemodel(df_sorted2, 2)
y_pred_S3010LL5_t2

In [ ]:
y_pred_S3010LL5_t3 = basemodel(df_sorted2, 3)
y_pred_S3010LL5_t3

In [ ]:
y_pred_S3010LL5_t4 = basemodel(df_sorted2, 4)
y_pred_S3010LL5_t4

test basemodel ZJV3BYAY


In [ ]:
y_pred_ZJV3BYAY_t2 = basemodel(df_sorted1, 2)
y_pred_ZJV3BYAY_t2


In [ ]:

y_pred_ZJV3BYAY_t3 = basemodel(df_sorted1, 3)
y_pred_ZJV3BYAY_t3
